<a href="https://colab.research.google.com/github/lanchuhuong/Food-Recognition/blob/main/TransferlearningV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import KFold
pd.set_option('display.max_columns', 50)
import re
import timeit
import random
random.seed(3)
import seaborn as sns
from tqdm.notebook import tqdm
from pathlib import Path
from sklearn.metrics import r2_score
import os
import random
import datetime
import re
import math
import logging
from collections import OrderedDict
import multiprocessing
import numpy as np
import skimage.transform
import tensorflow as tf
import keras
import keras.backend as K
import keras.layers as KL
import keras.engine as KE
import keras.models as KM
from keras.callbacks import EarlyStopping

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 1.7 MB 6.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 58 kB 3.0 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72857 sha256=36472f4b766e4a7d4b1288152b4a921da9b3b22483fe4237e2150e03561023e1
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle


In [ ]:
!mkdir ~/.kaggle
!echo '{"username":"lanchu","key":"004e7e514e204a75f54db6f58796d439"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c food-recognition-challenge-2021 -p data

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 97% 681M/699M [00:07<00:00, 107MB/s]
100% 699M/699M [00:08<00:00, 91.3MB/s]


In [ ]:
!unzip data/food-recognition-challenge-2021.zip -d data/

In [ ]:
labels = pd.read_csv('data/train_labels.csv').set_index('img_name')

In [ ]:
labels = labels.to_dict()
labels = labels[list(labels.keys())[0]]

print(f'lenght of dictionary: {len(labels)}')

df = pd.DataFrame(list(labels.items()))
df.columns = ['images', 'labels']
df = df.astype({'labels': str})
# Display the occurance of each class
df.groupby('labels').count()


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

train_df, validation_df = train_test_split(df, test_size=0.2)
print(f'train_df: {train_df.shape}')
print(f'validation_df: {validation_df.shape}')

train_df: (24489, 2)
validation_df: (6123, 2)


In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=15,
    height_shift_range=0.1,
    width_shift_range=0.1,
    shear_range=0.01,
    zoom_range=[0.9, 1.25],
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='reflect')

validation_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='data/train_set/train_set',
        x_col="images",
        y_col="labels",
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_dataframe(
        dataframe=validation_df,
        directory='data/train_set/train_set',
        x_col="images",
        y_col="labels",
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')

Found 24489 validated image filenames belonging to 80 classes.
Found 6123 validated image filenames belonging to 80 classes.


In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout, Flatten
from tensorflow.keras.optimizers import SGD



In [ ]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape = (256,256,3))

# baseModel = InceptionV3(weights="imagenet", include_top=False,
#     input_tensor=Input(shape=(256, 256, 3)))
# construct the head of the model that will be placed on top of the
# # the base model
# headModel = baseModel.output
# headModel = Flatten(name="flatten")(headModel)
# headModel = Dense(512, activation="relu")(headModel)
# headModel = Dropout(0.5)(headModel)
# headModel = Dense(80, activation="softmax")(headModel)
# # place the head FC model on top of the base model (this will become
# # the actual model we will train)

# model = Model(inputs=baseModel.input, outputs=headModel)

output = base_model.output

output = Flatten(name="flatten")(output)
predictions = Dense(80, activation='softmax')(output) #the output would be between 0-1 => confidence level , the output of the base layer
model = Model(inputs=base_model.input, outputs=predictions)
              
for layer in base_model.layers:
    layer.trainable = False

opt = SGD(learning_rate=1e-4, momentum=0.9)

model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

step_size_train=train_generator.n//train_generator.batch_size
step_size_val=validation_generator.n//train_generator.batch_size

history = model.fit(x=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=8,
                    validation_data = validation_generator,
                    validation_steps = step_size_val)

87924736/87910968 [==============================] - 1s 0us/step
Epoch 1/8
765/765 [==============================] - 534s 678ms/step - loss: 3.2417 - accuracy: 0.2491 - val_loss: 2.5799 - val_accuracy: 0.3835
Epoch 2/8
765/765 [==============================] - 521s 680ms/step - loss: 2.4680 - accuracy: 0.3931 - val_loss: 2.3913 - val_accuracy: 0.4203
Epoch 3/8
765/765 [==============================] - 522s 682ms/step - loss: 2.2218 - accuracy: 0.4421 - val_loss: 2.3424 - val_accuracy: 0.4357
Epoch 4/8
765/765 [==============================] - 517s 676ms/step - loss: 2.0545 - accuracy: 0.4765 - val_loss: 2.2955 - val_accuracy: 0.4458
Epoch 5/8
765/765 [==============================] - 527s 689ms/step - loss: 1.9328 - accuracy: 0.5017 - val_loss: 2.2607 - val_accuracy: 0.4580
Epoch 6/8
765/765 [==============================] - 536s 700ms/step - loss: 1.8295 - accuracy: 0.5245 - val_loss: 2.2685 - val_accuracy: 0.4542
Epoch 7/8
765/765 [==============================] - 530s 692ms/s